In [1]:
import numpy as np
import pandas as pd

#set cd to data folder with QC'ed rides
% cd /Users/fineiskid/Desktop/Access_Analysis_Rproject_local/data/


/Users/fineiskid/Desktop/Access_Analysis_Rproject_local/data


In [2]:
data = pd.read_csv("single_clean_day.csv")

In [3]:
data["ServiceDate"] = pd.to_datetime(data["ServiceDate"], format ='%Y-%m-%d %H:%M:%S', unit = "D")

In [4]:
def add_TimeWindows(data, windowsz):
    '''calculate time windows (pickup and dropoff)
        from SchTime and ETA.
        data is subsetted schedule data from a day.
        windowsz is size of pickup/dropoff window in seconds'''

    etas = data["ETA"]
    schtime =data["SchTime"]
    schtime[schtime<0] = np.nan
    data["PickupStart"] = 0; data["PickupEnd"] = 0
    data["DropoffStart"] = 0; data["DropoffEnd"] = 0
    for x in range(0, len(etas)):

        #make dropoff window when there's no required drop off time
        if (data["Activity"].iloc[x] == 1) & (data["ReqLate"].iloc[x] <0):
            data["DropoffStart"].iloc[x] = data["ETA"].iloc[x]-3600
            data["DropoffEnd"].iloc[x] = data["ETA"].iloc[x]+3600
        
        #make dropoff window when there IS a required drop off time: 1hr before ReqLate time
        if (data["Activity"].iloc[x] == 1) & (data["ReqLate"].iloc[x] >0):
            data["DropoffStart"].iloc[x] = data["ETA"].iloc[x]-3600
            data["DropoffEnd"].iloc[x] = data["ReqLate"].iloc[x]  
        
        #schtime is in the middle of the pick up window
        if data["Activity"].iloc[x] == 0:
            data["PickupStart"].iloc[x] = schtime.iloc[x]-(windowsz/2)
            data["PickupEnd"].iloc[x] = schtime.iloc[x]+(windowsz/2)

    return data

In [5]:
# SAVE SINGLE DAY'S RIDES WITH TIME WINDOWS
data = add_TimeWindows(data, 60*30)
data.to_csv(path = "/Users/fineiskid/Desktop/single_day_TimeWindows.csv", index = False)

'ServiceDate,Run,ProviderId,EvOrder,EvId,ReqTime,SchTime,ReqLate,Activity,ETA,DwellTime,StreetNo,OnStreet,City,LON,LAT,BookingId,SchedStatus,SubtypeAbbr,FundingSourceId1,PassOn,SpaceOn,PassOff,SpaceOff,ClientId,MobAids,PickupStart,PickupEnd,DropoffStart,DropoffEnd\n2014-04-14 00:00:00,680SEB,6,1,1093,-1,46800.0,-1,4,45900,0,8100.0,8TH AVE S,seattle,-122.32,47.53,,,,,,,,,0,,0.0,0.0,0,0\n2014-04-14 00:00:00,680SEB,6,5555559,19327,47700,47700.0,-1,0,47820,180,1200.0,12TH AVE S,seattle,-122.31,47.59,18032303.0,1.0,DEM,2.0,CLI1,AM1,,,106121,"CA,LI",46800.0,48600.0,0,0\n2014-04-14 00:00:00,680SEB,6,22222264,19325,50400,50400.0,-1,0,49354,180,14300.0,1ST AVE S,burien,-122.33,47.47,18032273.0,1.0,DEM,2.0,CLI1,"AM1,PK1",,,33094,"AP,CA,LI,PK",49500.0,51300.0,0,0\n2014-04-14 00:00:00,680SEB,6,29155540,19315,-1,,-1,1,50497,120,15024.0,29TH AVE S,seatac,-122.29,47.46,18032303.0,1.0,DEM,2.0,,,CLI1,AM1,106121,"CA,LI",0.0,0.0,46897,54097\n2014-04-14 00:00:00,680SEB,6,39462941,19215,52200,52200.0,-1,0,

In [6]:
def get_URIDs(data, broken_Run, resched_init_time):
    '''get unscheduled request id's from broken bus,
        based on when we're allowed to first start rescheduling.
        resched_init_time is in seconds, marks the point in time we can begin considering reinserting new requests.
        broken_Run is number of run that breaks
        data is today's scheduling data

        RETURN: list of URIDs'''
    
    #all rides that exist past time we're allowed to begin rescheduling
    leftover = data[data["ETA"] >= resched_init_time]
    leftover = leftover[(leftover["Activity"] != 6) & (leftover["Activity"] != 16) & (leftover["Activity"] != 3)]
    
    #rides that were scheduled to be on broken bus past resched_init_time
    pickmeup = leftover[leftover["Run"]==broken_Run]
    clients = pickmeup["ClientId"].unique()
    clients = clients[~(np.isnan(clients))]
    rmClients = []
    
    #remove people who would were scheduled to be on bus before resched_init_time
    for cli in clients:
        onoff = pickmeup[pickmeup["ClientId"]==cli]
        if onoff.shape[0] == 1:
            rmClients.append(cli)
    unsched = pickmeup[~pickmeup["ClientId"].isin(rmClients)]

    print("There are %s rides left to be scheduled on broken run %s" % (unsched.shape[0], broken_Run))

    class URID:
        def __init__(self, BookingId, Run, PickUpCoords, DropOffCoords, PickupStart, PickupEnd, DropoffStart, DropoffEnd, SpaceOn, MobAids):
            self.BookingId= BookingId
            self.Run = Run
            self.PickUpCoords = PickUpCoords
            self.DropOffCoords = DropOffCoords
            self.PickupStart = PickupStart
            self.PickupEnd = PickupEnd
            self.DropoffStart = DropoffStart
            self.DropoffEnd = DropoffEnd
            self.SpaceOn = SpaceOn
            self.MobAids = MobAids


    diffIDs = unsched.BookingId.unique()
    saveme = []

    #save separate URID's in a list
    for ID in diffIDs:
        temp = URID(BookingId = ID,
            Run = broken_Run,
            PickUpCoords = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["LAT", "LON"]].iloc[0,],
            DropOffCoords = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["LAT", "LON"]].iloc[1,],
            PickupStart = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["PickupStart"]].iloc[0,],
            PickupEnd = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["PickupEnd"]].iloc[0,],
            DropoffStart = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["DropoffStart"]].iloc[0,],
            DropoffEnd = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["DropoffEnd"]].iloc[0,],
            SpaceOn = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["SpaceOn"]].iloc[0,],
            MobAids = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["MobAids"]].iloc[0,])
        saveme.append(temp)

    return saveme

In [7]:
broken_Run = data.Run.unique()[0]
resched_init_time = 14*60*60
urids = get_URIDs(data, broken_Run, resched_init_time)

There are 10 rides left to be scheduled on broken run 680SEB


In [38]:
def get_busRuns(data, Run, resched_init_time):
    ''' take pd.DataFrame from add_Time_Windows.py and create busRun object for specified Run number,
    for all stops occurring at or after resched_init_time.

    RETURN: busRun object for specified Run.'''

    #subset based on matching Run number, and subset for stops only at or after resched_init_time
    print("Getting remaing rides for run %s" % Run)
    dataSub = data[(data["Run"] == Run)] # & (data["ETA"] >= resched_init_time)]
    #subset only the rides that aren't 6, 16, or 3:
    leave = dataSub.index.min()
    gas = dataSub[(dataSub["Activity"]==6)|(dataSub["Activity"]==16)|(dataSub["Activity"]==3)].index.min()
    busRun = dataSub.loc[leave:(gas-1)]

    return busRun

In [39]:
#Test get_busRuns subsetting routine
testRun = data.Run.unique()[2]
bus181SEB = get_busRuns(data, testRun, resched_init_time)

Getting remaing rides for run 181SEB


In [40]:
#list urid attributes:
urids[0].__dict__.keys()

['PickupEnd',
 'Run',
 'BookingId',
 'MobAids',
 'DropoffStart',
 'DropoffEnd',
 'PickUpCoords',
 'SpaceOn',
 'DropOffCoords',
 'PickupStart']

In [81]:
def time_overlap(Run_Schedule, URID, pickUpDropOff = True):
    '''URID: of class URID, has bookingId, pickUpLocation, dropOffLocation, etc.
        Run_Schedule: Schedule (pd.Data.Frame) of the run on which we're trying to insert the URID
        RETURN: pd.Data.Frame corresponding to nodes with overlapping pickup/dropoff windows'''

    #CURRENT ISSUES: RUNSCHEDULE SHOULD HAVE PRECEDING NODES, AT LEAST ONE PRIOR TO RESCHED_INIT_TIME

    if pickUpDropOff:
        Start = URID.PickupStart[0]
        End = URID.PickupEnd[0]
    else:
        Start = URID.DropoffStart[0]
        End = URID.DropoffEnd[0]
        
    crossover = []
    
    for jj in range(Run_Schedule.shape[0]):
        #Checking if a Run's PickupWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 0:
            PUE = Run_Schedule.PickupEnd.iloc[jj]; PUS = Run_Schedule.PickupStart.iloc[jj]
            #simple, unequal overlap
            if (PUE > Start) & (PUS < End):
                crossover.append(Run_Schedule.index[jj])
            # equal or strictly within [PUS, PUE]
            if (PUE <= End) & (PUS >= Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered by [PUS, PUE] and then some on both sides
            if (PUE > End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on left side
            if (PUE == End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on right side
            if (PUS == Start) & (PUE > End):
                crossover.append(Run_Schedule.index[jj])
                
        #Checking if a Run's DropoffWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 1:
            DOE = Run_Schedule.DropoffEnd.iloc[jj]; DOS = Run_Schedule.DropoffStart.iloc[jj]
            if (DOE > Start) & (DOS < End):
                crossover.append(Run_Schedule.index[jj])
            if (DOE <= End) & (DOS >= Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE > End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE == End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOS == Start) & (DOE > End):
                crossover.append(Run_Schedule.index[jj])
                
    #Get rid of cases that repeat themselves:
    crossover = set(crossover)
    indices = Run_Schedule.index
    firstnode = min(crossover); lastnode = max(crossover)
    #if this isn't the first or last run, add node that occurs right before
    #first time window overlap (lowerbound, upperbound nodes...)
    if min(crossover) != min(indices):
        crossover.add(indices[indices < firstnode].max())
    if max(crossover) != max(indices):
        crossover.add(indices[indices > lastnode].min())

    print("Need to service URID within %s sec to %s sec" % (Start, End))
    print("These indices of Run_Schedule will need\nto have distances calculated: %s" % crossover)

    return Run_Schedule.loc[crossover]


pickUpDropOff = True; URID = urids[0]; Run_Schedule = bus181SEB

In [82]:
from haversine import haversine
inserts = time_overlap(Run_Schedule, URID, pickUpDropOff)
overlap_LAT = inserts.LAT.tolist()
overlap_LON = inserts.LON.tolist()
if pickUpDropOff:
    URID_loc = ([URID.PickUpCoords["LAT"], URID.PickUpCoords["LON"]])
else:
    URID_loc = ([URID.DropOffCoords["LAT"], URID.PickUpCoords["LON"]])

Need to service URID within 51300.0 sec to 53100.0 sec
These indices of Run_Schedule will need
to have distances calculated: set([49, 50, 51, 52, 53, 54, 55, 58, 59, 60])


In [83]:
for k in range(len(overlap_LAT)):
    point = (overlap_LAT[k], overlap_LON[k])
    dist = haversine(point, URID_loc, miles=True)
    print(dist)

19.9223767133
19.1130045853
18.8151337871
23.8160392766
22.3902177184
22.3902177184
20.4168699431
18.5852218387
19.827313131
15.5833036725


In [84]:
inserts

,ServiceDate,Run,ProviderId,EvOrder,EvId,ReqTime,SchTime,ReqLate,Activity,ETA,...,PassOn,SpaceOn,PassOff,SpaceOff,ClientId,MobAids,PickupStart,PickupEnd,DropoffStart,DropoffEnd
49,2014-04-14,181SEB,5,31719963,18205,48780,48780,-1,0,48519,...,CLI1,AM1,NaN,NaN,88327,NaN,47880,49680,0,0
50,2014-04-14,181SEB,5,33203106,1580,-1,NaN,-1,1,49072,...,NaN,NaN,CLI1,WX1,104920,NaN,0,0,45472,52672
51,2014-04-14,181SEB,5,35839826,18152,-1,NaN,52200,1,49851,...,NaN,NaN,CLI1,AM1,88327,NaN,0,0,46251,52200
52,2014-04-14,181SEB,5,43749996,6700,-1,NaN,-1,1,50927,...,NaN,NaN,CLI1,"WH1,0T1",110433,"0T,WA",0,0,47327,54527
53,2014-04-14,181SEB,5,53749985,19117,51300,51300,-1,0,51806,...,CLI1,WG1,NaN,NaN,118663,"H1,LI",50400,52200,0,0
54,2014-04-14,181SEB,5,56022726,18800,52200,52200,-1,0,52103,...,"CLI1,PCA1","AM1,WX1",NaN,NaN,127568,"AP,LI,WA",51300,53100,0,0
55,2014-04-14,181SEB,5,57727235,18685,-1,NaN,-1,1,53235,...,NaN,NaN,"CLI1,PCA1","AM1,WX1",127568,"AP,LI,WA",0,0,49635,56835
58,2014-04-14,181SEB,5,59110537,18759,-1,NaN,-1,1,55780,...,NaN,NaN,"CLI1,PCA1","AM2,A1,PK1",75416,"A,AP,N,PK,V",0,0,52180,59380
59,2014-04-14,181SEB,5,59245350,19195,-1,NaN,-1,1,56326,...,NaN,NaN,CLI1,WG1,118663,"H1,LI",0,0,52726,59926
60,2014-04-14,181SEB,5,59964447,18572,-1,NaN,-1,1,58260,...,NaN,NaN,CLI1,AM1,90548,"CA,LI,V1",0,0,54660,61860


In [192]:
import requests
def osrm (URID_location, lats, lons):
    #URID_location it's a list: [lat, lon]
    #lists for inbound and outbound matrices
    # outbound (from urid location to scheduled location)
    # and inbound (from scheduled location to urid location) 
    out_total_time = []
    out_start_points = []
    out_end_points = []
    in_total_time = []
    in_start_points = []
    in_end_points = []
    out_osrm_url = "http://router.project-osrm.org/viaroute?"
    in_osrm_url = "http://router.project-osrm.org/viaroute?"
    urid_LAT = URID_location[0]; urid_LON = URID_location[1]

    # outbound 
    for lat_cord,lon_cord in zip(lats[0:-1],lons[0:-1]): 
        out_route_url = out_osrm_url + "loc=" + str(urid_LAT) + "," + str(urid_LON) 
        out_route_url = out_route_url+ "&loc=" + str(lat_cord) + "," + str(lon_cord) + "&instructions=false"
        out_route_requests = requests.get(out_route_url)
        out_route_results = out_route_requests.json()
        out_total_time += [out_route_results[u'route_summary'][u'total_time']]
        out_start_points += [out_route_results[u'route_summary'][u'start_point']]
        out_end_points += [out_route_results[u'route_summary'][u'end_point']]

    # inbound
    for lat_cord,lon_cord in zip(lats[1:],lons[1:]): 
        in_route_url = in_osrm_url + "loc=" + str(lat_cord) + "," + str(lon_cord) 
        in_route_url = in_route_url+ "&loc=" + str(urid_LAT) + "," + str(urid_LON) + "&instructions=false"
        in_route_requests = requests.get(in_route_url)
        in_route_results = in_route_requests.json()
        in_total_time += [in_route_results[u'route_summary'][u'total_time']]
        in_start_points += [in_route_results[u'route_summary'][u'start_point']]
        in_end_points += [in_route_results[u'route_summary'][u'end_point']]

    a = np.array([in_total_time]); b = np.array([out_total_time])
    ret = np.concatenate((a, b), axis = 0)
    total_route_df = pd.DataFrame(data = ret.T, columns=['inbound [1:]', 'outbound[0:-1]'])
    
    return(total_route_df)


In [193]:
import time
lats = inserts.LAT.tolist(); lons = inserts.LON.tolist()
uridLoc = [URID.PickUpCoords.LAT, URID.PickUpCoords.LON]
start = time.clock()
time_matrix = osrm(uridLoc, lats, lons)
elapsed = time.clock()-start
print("API call took %s sec" % elapsed)

API call took 0.075895 sec


In [195]:
time_matrix

,inbound [1:],outbound[0:-1]
0,2162,1939
1,2043,2013
2,2515,1873
3,2415,2336
4,2415,2300
5,2182,2300
6,2048,2050
7,2200,1879
8,1746,2030


[[2162, 2043, 2515, 2415, 2415, 2182, 2048, 2200, 1746]]